# 🛒 Análisis de Canasta de Mercado: Retail en España

Mi primer proyecto de análisis de datos con Python.

## Objetivos:
1. Implementar el algoritmo Apriori desde cero
2. Descubrir patrones de compra frecuentes  
3. Visualizar las reglas de asociación más relevantes

## 1. Importación de librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Librerías importadas correctamente")

## 2. Generación del Dataset

Creamos transacciones realistas basadas en categorías de productos españoles.

In [ ]:
def generar_dataset_transacciones(n_transacciones=8000, random_state=42):
    """Genera dataset de transacciones de supermercado español."""
    
    np.random.seed(random_state)
    
    categorias = {
        'Alimentos_bebidas': ['Leche entera', 'Leche desnatada', 'Yogur natural', 
                              'Queso fresco', 'Mantequilla', 'Pan de barra', 
                              'Pan integral', 'Huevos', 'Aceite oliva'],
        'Carne_pescado': ['Pollo entero', 'Filete ternera', 'Jamón ibérico', 
                         'Chorizo', 'Pechuga pavo', 'Merluza', 'Salmón', 
                         'Gambas', 'Atún enlatado'],
        'Frutas_verduras': ['Manzanas', 'Plátanos', 'Naranjas', 'Tomates', 
                           'Lechuga', 'Cebolla', 'Patatas', 'Zanahorias', 'Pimientos'],
        'Bebidas': ['Agua mineral', 'Refresco cola', 'Zumo naranja', 
                   'Cerveza', 'Vino tinto', 'Vino blanco', 'Café molido'],
        'Despensa': ['Arroz', 'Pasta', 'Azúcar', 'Sal', 'Conserva tomate', 
                    'Legumbres', 'Harina', 'Cereales'],
        'Congelados': ['Pizza congelada', 'Helado vainilla', 'Verduras congeladas', 
                      'Pescado congelado', 'Croquetas'],
        'Higiene_hogar': ['Papel higiénico', 'Champú', 'Gel de ducha', 
                         'Detergente', 'Limpiador multiusos', 'Suavizante'],
        'Dulces_snacks': ['Galletas chocolate', 'Chocolate negro', 'Turron', 
                         'Miel', 'Patatas fritas', 'Frutos secos']
    }
    
    reglas = {
        'Vino tinto': {'asociados': ['Queso fresco', 'Jamón ibérico', 'Pan de barra'], 'prob': 0.65},
        'Cerveza': {'asociados': ['Jamón ibérico', 'Gambas', 'Patatas fritas'], 'prob': 0.60},
        'Pan de barra': {'asociados': ['Mantequilla', 'Jamón ibérico', 'Tomates'], 'prob': 0.55},
        'Pasta': {'asociados': ['Tomates', 'Queso fresco', 'Aceite oliva'], 'prob': 0.70},
        'Arroz': {'asociados': ['Pollo entero', 'Tomates', 'Aceite oliva'], 'prob': 0.60},
        'Café molido': {'asociados': ['Leche entera', 'Azúcar', 'Galletas chocolate'], 'prob': 0.50},
        'Gambas': {'asociados': ['Vino blanco', 'Pan de barra'], 'prob': 0.55},
        'Pizza congelada': {'asociados': ['Refresco cola', 'Helado vainilla'], 'prob': 0.45}
    }
    
    transacciones = []
    
    for _ in range(n_transacciones):
        n_categorias = np.random.randint(2, 6)
        cats_seleccionadas = np.random.choice(list(categorias.keys()), n_categorias, replace=False)
        
        carrito = []
        for cat in cats_seleccionadas:
            n_productos = np.random.randint(1, 4)
            productos = np.random.choice(categorias[cat], min(n_productos, len(categorias[cat])), replace=False)
            carrito.extend(productos)
        
        for producto_base, info in reglas.items():
            if producto_base in carrito and np.random.random() < info['prob']:
                asociados = np.random.choice(info['asociados'], np.random.randint(1, 3), replace=False)
                carrito.extend(asociados)
        
        transacciones.append(list(set(carrito)))
    
    return transacciones, categorias

print("🔄 Generando dataset...")
transacciones, categorias = generar_dataset_transacciones()

print(f"✅ Dataset generado:")
print(f"   - Transacciones: {len(transacciones):,}")
print(f"   - Productos únicos: {len(set([item for sublist in transacciones for item in sublist]))}")
print(f"   - Media productos/transacción: {np.mean([len(t) for t in transacciones]):.1f}")

## 3. Implementación del Algoritmo Apriori

In [ ]:
class AprioriAnalyzer:
    """Implementación del algoritmo Apriori para análisis de canasta de mercado."""
    
    def __init__(self, transactions, min_support=0.03, min_confidence=0.40):
        self.transactions = transactions
        self.n_transactions = len(transactions)
        self.min_support = min_support
        self.min_confidence = min_confidence
        self.frequent_itemsets = {}
        self.rules = []
        
    def _get_itemsets(self, k):
        """Obtiene todos los itemsets de tamaño k."""
        itemsets = Counter()
        for transaction in self.transactions:
            if len(transaction) >= k:
                for combo in combinations(transaction, k):
                    itemsets[frozenset(combo)] += 1
        return itemsets
    
    def _filter_by_support(self, itemsets):
        """Filtra itemsets por soporte mínimo."""
        return {itemset: count for itemset, count in itemsets.items() 
                if count / self.n_transactions >= self.min_support}
    
    def find_frequent_itemsets(self, max_k=2):
        """Encuentra itemsets frecuentes hasta tamaño max_k."""
        print(f"🔍 Buscando itemsets frecuentes (support >= {self.min_support*100}%)...")
        
        for k in range(1, max_k + 1):
            itemsets = self._get_itemsets(k)
            frequent = self._filter_by_support(itemsets)
            
            if not frequent:
                break
                
            self.frequent_itemsets[k] = frequent
            print(f"   k={k}: {len(frequent)} itemsets encontrados")
        
        return self.frequent_itemsets
    
    def generate_rules(self):
        """Genera reglas de asociación a partir de itemsets frecuentes."""
        if 2 not in self.frequent_itemsets:
            raise ValueError("No hay itemsets de tamaño 2. Reduce min_support.")
        
        print(f"📈 Generando reglas (confidence >= {self.min_confidence*100}%)...")
        
        frequent_1 = self.frequent_itemsets[1]
        frequent_2 = self.frequent_itemsets[2]
        
        rules = []
        
        for itemset_2, count_2 in frequent_2.items():
            items = list(itemset_2)
            item_a, item_b = items[0], items[1]
            
            support = count_2 / self.n_transactions
            
            # Regla: A -> B
            count_a = frequent_1.get(frozenset([item_a]), 0)
            confidence_a_b = count_2 / count_a if count_a > 0 else 0
            count_b = frequent_1.get(frozenset([item_b]), 0)
            support_b = count_b / self.n_transactions
            lift_a_b = confidence_a_b / support_b if support_b > 0 else 0
            
            if confidence_a_b >= self.min_confidence:
                rules.append({
                    'antecedent': item_a,
                    'consequent': item_b,
                    'support': support,
                    'confidence': confidence_a_b,
                    'lift': lift_a_b,
                    'count': count_2
                })
            
            # Regla: B -> A
            confidence_b_a = count_2 / count_b if count_b > 0 else 0
            support_a = count_a / self.n_transactions
            lift_b_a = confidence_b_a / support_a if support_a > 0 else 0
            
            if confidence_b_a >= self.min_confidence:
                rules.append({
                    'antecedent': item_b,
                    'consequent': item_a,
                    'support': support,
                    'confidence': confidence_b_a,
                    'lift': lift_b_a,
                    'count': count_2
                })
        
        self.rules_df = pd.DataFrame(rules)
        if not self.rules_df.empty:
            self.rules_df = self.rules_df.sort_values('lift', ascending=False)
        
        print(f"✅ {len(self.rules_df)} reglas generadas")
        return self.rules_df

# Ejecutar análisis
analyzer = AprioriAnalyzer(transacciones, min_support=0.03, min_confidence=0.40)
frequent_itemsets = analyzer.find_frequent_itemsets(max_k=2)
rules_df = analyzer.generate_rules()

print("\n🏆 TOP 5 REGLAS POR LIFT:")
print("-" * 60)
for idx, rule in rules_df.head(5).iterrows():
    print(f"{rule['antecedent']} → {rule['consequent']}")
    print(f"   Soporte: {rule['support']:.1%} | Confianza: {rule['confidence']:.1%} | Lift: {rule['lift']:.2f}")
    print()

## 4. Visualizaciones

In [ ]:
def plot_market_basket_analysis(rules_df, transactions, categorias, output_path='images/'):
    """Genera visualizaciones del análisis."""
    
    import os
    os.makedirs(output_path, exist_ok=True)
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))
    fig.suptitle('Análisis de Canasta de Mercado: Retail en España', 
                 fontsize=16, fontweight='bold', y=0.98)
    
    # 1. Top reglas por Lift
    ax1 = axes[0, 0]
    top_rules = rules_df.head(10)
    y_pos = np.arange(len(top_rules))
    colors = plt.cm.RdYlGn(top_rules['lift'] / top_rules['lift'].max())
    
    bars = ax1.barh(y_pos, top_rules['lift'], color=colors)
    ax1.set_yticks(y_pos)
    ax1.set_yticklabels([f"{row['antecedent'][:15]}...→{row['consequent'][:15]}..." 
                         for _, row in top_rules.iterrows()], fontsize=9)
    ax1.set_xlabel('Lift', fontweight='bold')
    ax1.set_title('Top 10 Reglas por Lift', fontweight='bold', pad=20)
    ax1.grid(axis='x', alpha=0.3)
    
    # 2. Scatter: Soporte vs Confianza
    ax2 = axes[0, 1]
    scatter = ax2.scatter(rules_df['support'] * 100, rules_df['confidence'] * 100, 
                         s=rules_df['lift'] * 50, c=rules_df['lift'], 
                         cmap='viridis', alpha=0.6, edgecolors='black', linewidth=0.5)
    ax2.set_xlabel('Soporte (%)', fontweight='bold')
    ax2.set_ylabel('Confianza (%)', fontweight='bold')
    ax2.set_title('Reglas: Soporte vs Confianza\n(Tamaño = Lift)', fontweight='bold', pad=20)
    ax2.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax2, label='Lift')
    
    # 3. Distribución por categorías
    ax3 = axes[1, 0]
    categoria_count = {}
    for cat, productos in categorias.items():
        count = sum(1 for t in transactions for p in productos if p in t)
        categoria_count[cat] = count
    
    cat_df = pd.DataFrame(list(categoria_count.items()), columns=['Categoría', 'Frecuencia'])
    cat_df = cat_df.sort_values('Frecuencia', ascending=True)
    
    colors_cat = plt.cm.Set3(np.linspace(0, 1, len(cat_df)))
    ax3.barh(cat_df['Categoría'], cat_df['Frecuencia'], color=colors_cat)
    ax3.set_xlabel('Frecuencia en Transacciones', fontweight='bold')
    ax3.set_title('Distribución por Categorías', fontweight='bold', pad=20)
    ax3.grid(axis='x', alpha=0.3)
    
    # 4. Matriz de co-ocurrencia
    ax4 = axes[1, 1]
    top_products = rules_df.head(8)
    products_list = list(set(top_products['antecedent'].tolist() + 
                            top_products['consequent'].tolist()))
    
    cooccur_matrix = pd.DataFrame(0, index=products_list, columns=products_list)
    
    for _, rule in top_products.iterrows():
        cooccur_matrix.loc[rule['antecedent'], rule['consequent']] = rule['lift']
    
    sns.heatmap(cooccur_matrix, annot=True, fmt='.2f', cmap='YlOrRd', 
                ax=ax4, cbar_kws={'label': 'Lift'})
    ax4.set_title('Matriz de Asociación (Lift)', fontweight='bold', pad=20)
    ax4.set_xlabel('Producto Consecuente', fontweight='bold')
    ax4.set_ylabel('Producto Antecedente', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(f'{output_path}market_basket_analysis.png', dpi=300, bbox_inches='tight')
    print(f"✅ Gráfico guardado en {output_path}market_basket_analysis.png")
    plt.show()

# Generar visualizaciones
plot_market_basket_analysis(rules_df, transacciones, categorias)

## 5. Resumen

In [ ]:
print("="*60)
print("📊 RESUMEN EJECUTIVO")
print("="*60)
print(f"Transacciones analizadas: {len(transacciones):,}")
print(f"Reglas descubiertas: {len(rules_df)}")
print(f"Mejor regla por Lift: {rules_df.iloc[0]['antecedent']} → {rules_df.iloc[0]['consequent']} ({rules_df.iloc[0]['lift']:.2f})")
print(f"Mejor regla por Confianza: {rules_df.sort_values('confidence', ascending=False).iloc[0]['antecedent']} → {rules_df.sort_values('confidence', ascending=False).iloc[0]['consequent']} ({rules_df.sort_values('confidence', ascending=False).iloc[0]['confidence']:.1%})")
print("="*60)